# Explore the data

# !!!!!!!!!!!!!!!!
# Looks like we may be able to use household data and household ids to make generalizations... and use more data 
# !!!!!!!!!!!!!!!!

Maybe a predictor based directly on the household id?...

In [21]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


In [ ]:
# Additional covariate called family size. to USE more data. 
# How to use more data by groupping poverty_scores on household?
# within same household
# add a psu_hh uui as a VARIABLE to tell the model "predict something similar for members of the same family". It would be difficult, there is a lot.
# stacking? what is stacking?

In [25]:
train_ed = pd.read_csv('./training/education.csv') # education
train_hh = pd.read_csv('./training/household.csv') # household
test_ed = pd.read_csv('edu_test.csv')
test_hh = pd.read_csv('hh_test.csv')

poverty = pd.read_csv('./training/poverty.csv') # poverty/answers

# df_pov_scores['hh'] = df_pov_scores['uid'].apply(lambda uid : int(uid.split('_')[1]) )
# df_ed

In [44]:
### Prepare Ed data
def prepare_df(df, suffix:str):
    uids = df['psu'].astype(str) + "_"  + df['hh'].astype(str) + "_" + df['idcode'].astype(str) 
    df.columns = [suffix + "_" + col.capitalize() for col in df.columns]
    df.insert(0, 'uid', uids)
    return df

test_ed = prepare_df(test_ed, 'ED')
test_hh = prepare_df(test_hh, 'HH')
train_ed = prepare_df(train_ed, 'ED')
train_hh = prepare_df(train_hh, 'HH')

In [29]:
### compute poverty scores
for i in range(1,11):
    col = 'subjective_poverty_'+ str(i)
    poverty.loc[poverty[col]==1, 'poverty_score'] = i

poverty['uid'] = poverty['psu_hh_idcode']
poverty = poverty[['uid', 'poverty_score']]



In [133]:
spl = '30_2_2'.split('_')
'_'.join([spl[0], spl[1]])

'30_2'

In [137]:
poverty['fid'] = poverty['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 


In [139]:
poverty

,uid,poverty_score,fid
0,30_8_1,4.0,30_8
1,194_1_2,1.0,194_1
2,224_6_1,3.0,224_6
3,323_10_1,5.0,323_10
4,428_10_1,4.0,428_10
...,...,...,...
5332,571_8_1,3.0,571_8
5333,601_5_1,4.0,601_5
5334,782_1_1,2.0,782_1
5335,606_3_1,5.0,606_3


In [46]:
# Explore test data:
print(test_ed.shape)
print(test_hh.shape)
assert(test_hh[test_hh['uid'].isin(test_ed['uid'])].shape[0] == 1334)
## No issues here.

(1334, 70)
(1334, 27)


# Group #1 


In [80]:
# Claim: Those who answer No = 2 for ED_Q3 are done with the ED survey. 
group1 = test_ed['ED_Q03'].eq(2)
# This means they are a group. In a tree, a split would go here. 
test_ed[group1].to_csv('./group1/test_group1.csv', index=False)


group1 = train_ed['ED_Q03'].eq(2)
# This means they are a group. In a tree, a split would go here. 
train_ed[group1].to_csv('./group1/train_group1.csv', index=False)

## Also add the scores.



,uid,ED_Psu,ED_Hh,ED_Idcode,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,...,ED_Q57,ED_Q58,ED_Q59,ED_Q60,ED_Q61,ED_Q62,ED_Q63,ED_Q64,ED_Q65,ED_Q66
0,2_8_3,2,8,3,1,1,1,2.0,3.0,3.0,...,2.0,NaN,2.0,NaN,2.0,NaN,NaN,2.0,NaN,2.0
1,3_8_4,3,8,4,1,1,1,2.0,2.0,2.0,...,2.0,NaN,2.0,NaN,2.0,NaN,NaN,2.0,NaN,2.0
2,6_5_3,6,5,3,1,1,1,2.0,3.0,3.0,...,2.0,NaN,2.0,NaN,2.0,NaN,NaN,2.0,NaN,4.0
3,7_7_3,7,7,3,1,1,1,6.0,1.0,3.0,...,2.0,NaN,2.0,NaN,2.0,NaN,NaN,2.0,NaN,6.0
4,16_2_4,16,2,4,1,1,1,5.0,3.0,2.0,...,2.0,NaN,2.0,NaN,2.0,NaN,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22401,72_1_4,72,1,4,1,1,1,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22402,322_7_3,322,7,3,1,1,1,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22403,808_10_3,808,10,3,1,1,1,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22404,377_1_2,377,1,2,1,1,1,1.0,8.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
group2_temp = train_ed.loc[~group1]
group2_temp.to_csv('./group2/train_group2.csv', index=False)


In [88]:
 # There might be a group 3 which splits from group2 at Q14? everything after 20 is 0?
group3 = group2_temp['ED_Q14'].is(2)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
22401    2.0
22402    2.0
22403    2.0
22404    2.0
22405    2.0
Name: ED_Q14, Length: 21513, dtype: float64

# Households

before you even split into groups, try your hand at usign households to "guess" more people's poverty score and increase the amount of data used

In [117]:
# The first division is by "primary sampling unit." It's like the first sample. 
# Members from the same family are in the same sample. So psh_hh is basically the family id.
# idcode is then the id of the individual..

# maybe you could average family responses into the observation with the poverty score... if poverty scores only has about one person from each family and there are more in the full data.


psu1 = train_ed[train_ed['ED_Psu']==1]
print(psu1.shape)
family_1_12 = psu1[psu1['ED_Hh']== 12]
family_1_12

(17, 70)


,uid,ED_Psu,ED_Hh,ED_Idcode,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,...,ED_Q57,ED_Q58,ED_Q59,ED_Q60,ED_Q61,ED_Q62,ED_Q63,ED_Q64,ED_Q65,ED_Q66
909,1_12_3,1,12,3,1,1,1,2.0,1.0,2.0,...,2.0,NaN,2.0,NaN,1.0,5.0,12.0,2.0,NaN,5.0
1029,1_12_2,1,12,2,1,1,1,2.0,1.0,2.0,...,2.0,NaN,2.0,NaN,1.0,5.0,12.0,2.0,NaN,5.0
17444,1_12_1,1,12,1,1,1,1,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
cond = poverty['fid'].eq("1_12")
poverty[cond]
# we could, for instance, set the poverty_scores of all three members of the family equal to each other. 
# HOWEVER, the questions is: how rich do YOU THINK YOU are? a kid with no allowance may think he is starving. 
# Even so, I think the variable: How rich does your family think you are? is good for training. 

,uid,poverty_score,fid
3841,1_12_1,4.0,1_12


In [147]:
# No repeated fid's in the poverty_df. 
# So no two members of the family have an opinion on how poor/rich they are. 
# but if you are a kid and your dad says youre poor, you probably think you're poor??? there's so many vars
frequencies = poverty['fid'].value_counts()
# frequencies

fid
30_8     1
801_2    1
542_8    1
802_5    1
440_6    1
        ..
799_2    1
680_4    1
779_3    1
427_3    1
450_4    1
Name: count, Length: 5337, dtype: int64

In [161]:
# are there any of the fid's in the test data also in the poverty scores? probably not, it would be super nice.
test_hh['fid'] = test_hh['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 
poverty[poverty['uid'].isin(test_hh['fid'])] # Nope

,uid,poverty_score,fid


In [94]:
# why are there some entries with the SAME idcodes and HOUSEHOLD to each other!????
train_ed[train_ed['hh']==10] ### psu_hh .... first houses into groups psu is first level division.
# could we continuate? psu_hh is family
hh10 = df_pov_scores[df_pov_scores['hh']==10]
hh10['poverty_score'].describe()

KeyError: 'hh'

In [ ]:
plt.hist(hh10['poverty_score'], bins=5, color='blue', edgecolor='black', alpha=0.7)